In [1]:
import argparse
import os
import numpy as np
import math

import torchvision
import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

import matplotlib.pyplot as plt

In [2]:
os.makedirs("images", exist_ok=True)

# parser = argparse.ArgumentParser()
# parser.add_argument("--n_epochs", type=int, default=200, help="number of epochs of training")
# parser.add_argument("--batch_size", type=int, default=64, help="size of the batches")
# parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")
# parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
# parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")
# parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")
# parser.add_argument("--img_size", type=int, default=32, help="size of each image dimension")
# parser.add_argument("--channels", type=int, default=1, help="number of image channels")
# parser.add_argument("--sample_interval", type=int, default=400, help="interval between image sampling")
# opt = parser.parse_args()
# print(opt)

n_epochs = 200
batch_size = 20
lr = 0.0002
b1 = .5
b2 = .999
n_cpu = 8
latent_dim = 2
img_size = 64
channels = 1
sample_interval = 25

cuda = True if torch.cuda.is_available() else False


def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.init_size = img_size // 4
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128 * self.init_size ** 2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh(),
        )

    def forward(self, z):
        out = self.l1(z)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            block = [nn.Conv2d(in_filters, out_filters, 3, 2, 1), nn.LeakyReLU(0.2, inplace=True), nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.model = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2 ** 4
        self.adv_layer = nn.Sequential(nn.Linear(128 * ds_size ** 2, 1), nn.Sigmoid())

    def forward(self, img):
        out = self.model(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)

        return validity


# Loss function
adversarial_loss = torch.nn.BCELoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
# os.makedirs("../../data/mnist", exist_ok=True)
# dataloader = torch.utils.data.DataLoader(
#     datasets.MNIST(
#         "../../data/mnist",
#         train=True,
#         download=True,
#         transform=transforms.Compose(
#             [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
#         ),
#     ),
#     batch_size=batch_size,
#     shuffle=True,
# )

# dataloader = torch.utils.data.DataLoader(
#     datasets.MNIST(
#         "../dataset",
#         train=True,
#         download=False,
#         transform=transforms.Compose(
#             [transforms.Resize(img_size), transforms.ToTensor(), transforms.Normalize([0.5], [0.5])]
#         ),
#     ),
#     batch_size=batch_size,
#     shuffle=True,
# )

class ImageFolderWithPaths(torchvision.datasets.ImageFolder):
    def __getitem__(self, index):
        return super(ImageFolderWithPaths, self).__getitem__(index) + (self.imgs[index][0],)

train_dataset = ImageFolderWithPaths(
    root='../dataset',
    transform=transforms.Compose([
        transforms.Resize(img_size),
        transforms.CenterCrop(img_size),
        transforms.Grayscale(),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, ), std=(0.5, )),
    ])
)

dataloader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    num_workers=n_cpu,
    shuffle=True,
    drop_last=True
)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
# ----------
#  Training
# ----------

for epoch in range(n_epochs):
    for i, (imgs, _, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise as generator input
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))

        # Generate a batch of images
        gen_imgs = generator(z)

        # Loss measures generator's ability to fool the discriminator
        g_loss = adversarial_loss(discriminator(gen_imgs), valid)

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(real_imgs), valid)
        fake_loss = adversarial_loss(discriminator(gen_imgs.detach()), fake)
        d_loss = (real_loss + fake_loss) / 2

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, n_epochs, i, len(dataloader), d_loss.item(), g_loss.item())
        )
        
        if epoch % sample_interval == 0:
            save_image(gen_imgs.data[:25], "images/%d.png" % epoch, nrow=5, normalize=True)

[Epoch 0/200] [Batch 0/6] [D loss: 0.654978] [G loss: 0.663951]
[Epoch 0/200] [Batch 1/6] [D loss: 0.672951] [G loss: 0.673644]
[Epoch 0/200] [Batch 2/6] [D loss: 0.694147] [G loss: 0.645313]
[Epoch 0/200] [Batch 3/6] [D loss: 0.670460] [G loss: 0.712233]
[Epoch 0/200] [Batch 4/6] [D loss: 0.669932] [G loss: 0.699915]
[Epoch 0/200] [Batch 5/6] [D loss: 0.704084] [G loss: 0.708768]
[Epoch 1/200] [Batch 0/6] [D loss: 0.684897] [G loss: 0.709774]
[Epoch 1/200] [Batch 1/6] [D loss: 0.676491] [G loss: 0.680502]
[Epoch 1/200] [Batch 2/6] [D loss: 0.699827] [G loss: 0.683878]
[Epoch 1/200] [Batch 3/6] [D loss: 0.683793] [G loss: 0.649495]
[Epoch 1/200] [Batch 4/6] [D loss: 0.670246] [G loss: 0.673500]
[Epoch 1/200] [Batch 5/6] [D loss: 0.707621] [G loss: 0.692776]
[Epoch 2/200] [Batch 0/6] [D loss: 0.652587] [G loss: 0.734947]
[Epoch 2/200] [Batch 1/6] [D loss: 0.641094] [G loss: 0.716367]
[Epoch 2/200] [Batch 2/6] [D loss: 0.674695] [G loss: 0.786375]
[Epoch 2/200] [Batch 3/6] [D loss: 0.684

[Epoch 21/200] [Batch 2/6] [D loss: 0.683430] [G loss: 0.651015]
[Epoch 21/200] [Batch 3/6] [D loss: 0.666456] [G loss: 0.674765]
[Epoch 21/200] [Batch 4/6] [D loss: 0.677283] [G loss: 0.659808]
[Epoch 21/200] [Batch 5/6] [D loss: 0.681999] [G loss: 0.700275]
[Epoch 22/200] [Batch 0/6] [D loss: 0.700774] [G loss: 0.691508]
[Epoch 22/200] [Batch 1/6] [D loss: 0.695619] [G loss: 0.723607]
[Epoch 22/200] [Batch 2/6] [D loss: 0.671817] [G loss: 0.749697]
[Epoch 22/200] [Batch 3/6] [D loss: 0.689422] [G loss: 0.735658]
[Epoch 22/200] [Batch 4/6] [D loss: 0.733402] [G loss: 0.704083]
[Epoch 22/200] [Batch 5/6] [D loss: 0.697212] [G loss: 0.717228]
[Epoch 23/200] [Batch 0/6] [D loss: 0.675890] [G loss: 0.724160]
[Epoch 23/200] [Batch 1/6] [D loss: 0.677971] [G loss: 0.713279]
[Epoch 23/200] [Batch 2/6] [D loss: 0.675889] [G loss: 0.762412]
[Epoch 23/200] [Batch 3/6] [D loss: 0.686100] [G loss: 0.707674]
[Epoch 23/200] [Batch 4/6] [D loss: 0.646530] [G loss: 0.729909]
[Epoch 23/200] [Batch 5/6

In [5]:
# Explore latent space
from ipywidgets import interact

def sample(z1, z2):
    #z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], latent_dim))))
    z = np.zeros((1, latent_dim))
    z[0,0] = z1
    z[0,1] = z2
    z = Variable(Tensor(z))
    gen_imgs = generator(z)
    plt.imshow(gen_imgs.data[0, 0] * .5 + .5, cmap='gray')
    
interact(sample, z1=(-1, 1, .1), z2=(-1, 1, .1))

interactive(children=(FloatSlider(value=0.0, description='z1', max=1.0, min=-1.0), FloatSlider(value=0.0, desc…

<function __main__.sample(z1, z2)>